In [11]:
import numpy as np
import pandas as pd
from FinancialInstrument import FinancialInstrument
from ETF import ETF
from Future import Future
from datetime import date
from DateRanges import electionPeriodBoolsDF, e_year_ranges

class Portfolio:
    """
    Class representing a portfolio of financial instruments.
    """

    def __init__(self, instrument_weight_list):
        """
        Initialize a portfolio.

        Args:
            instrument_weight_list (FinancialInstrument, float): List of tuples where the the first element
                is the FinancialInstrument and the second element is the weight as a float.
        """
        self.instruments, self.weights = zip(*instrument_weight_list)
        self.weights = np.array(self.weights)
        self._validate_weights()
        self.start_date = (max(self.instruments, key=lambda instrument: instrument.get_date_range()[0])).get_date_range()[0]
        self.end_date = (min(self.instruments, key=lambda instrument: instrument.get_date_range()[1])).get_date_range()[1]
        for instrument in self.instruments:
            instrument.filter(self.start_date, self.end_date)
        log_returns_dict = {
            instrument.ticker: instrument.log_returns for instrument in self.instruments
        }
        self.full_asset_log_returns_df = pd.concat(log_returns_dict, axis=1)
        self.asset_log_returns_df = self.full_asset_log_returns_df
        self.portfolio_log_returns = self.asset_log_returns_df.dot(self.weights)

    def filter(self, startDate=date(1800, 1, 1), endDate=date(2100, 12, 31), period=0):
        filtered = self.full_asset_log_returns_df.loc[startDate:endDate]
        self.asset_log_returns_df = filtered
        merged_df = pd.merge(self.asset_log_returns_df, electionPeriodBoolsDF, left_index=True, right_index=True, how='inner')
        if period == 1:
            merged_df = merged_df[merged_df['In an Election Period']]
        elif period == -1:
            merged_df = merged_df[~merged_df['In an Election Period']]
        merged_df = merged_df.drop(columns=['In an Election Period'])
        self.asset_log_returns_df = merged_df
        
    def _validate_weights(self):
        """
        Validate that the sum of weights equals 1.
        """
        total_weight = sum(self.weights)
        if not np.isclose(total_weight, 1.0):
            raise ValueError(f"Portfolio weights must sum to 1. Current sum: {total_weight}")
    
    def change_weights(self, new_weights):
        self._validate_weights()
        self.weights = np.array(new_weights)
        self.portfolio_log_returns = self.asset_log_returns_df.dot(self.weights)
        return

    def expected_log_return(self):
        """
        Calculate the expected log return of the portfolio.

        Returns:
            float: The expected return of the portfolio.
        """
        total_log_return = 0
        for instrument, weight in zip(self.instruments, self.weights):
            total_log_return += weight * instrument.expected_log_return()
        return total_log_return

    def covariance_matrix(self):
        """
        Calculate the covariance matrix for the portfolio's instruments.

        Returns:
            np.ndarray: Covariance matrix of the portfolio's instruments.
        """
        return self.instruments[0].covariance_matrix(self.instruments[1:])
    
    def portfolio_variance(self):
        """
        Calculate the portfolio's variance.

        Returns:
            float: The portfolio variance.
        """
        return np.dot(self.weights.T, np.dot(self.covariance_matrix(), self.weights))
    
    def portfolio_volatility(self):
        """
        Calculate the portfolio's volatility.

        Returns:
            float: The portfolio volatility.
        """
        return np.sqrt(self.portfolio_variance())
    
    def calculate_beta(self, benchmark):
        total_beta = 0
        for instrument, weight in zip(self.instruments, self.weights):
            total_beta += weight * instrument.calculate_beta(benchmark)
        return total_beta
        

    def summary(self):
        """
        Print a summary of the portfolio.

        Returns:
            str: Summary of the portfolio.
        """
        summary_lines = [f"Instrument: {inst.ticker}, Weight: {weight}" 
                         for inst, weight in zip(self.instruments, self.weights)]
        return "\n".join(summary_lines) + f"\nExpected Log Return: {self.expected_log_return():.2}\n" \
                                          f"Portfolio Volatility: {self.portfolio_volatility():.2}"

In [12]:
# etfSPY = ETF("SPY")
# etfXLB = ETF("XLB")
# etfXLU = ETF("XLU")
# portfolio1 = Portfolio([(etfSPY, 0.25), (etfXLB, 0.65), (etfXLU, 0.10)])
# print(portfolio1.summary())

etfSPY = ETF("SPY")
print(etfSPY.summary())
print(etfSPY.log_returns)
print()
etfSPY.filter(startDate="2001-05-05")
print(etfSPY.log_returns)
print(etfSPY.summary())
print()
etfSPY.filter(period=-1)
print(etfSPY.log_returns)
print(etfSPY.summary())
print(e_year_ranges)

Type: ETF, Ticker: SPY, Period: All Periods, Volatility: 0.19828607421028432, First Date: 1998-12-23 00:00:00, Last Date: 2020-12-30 00:00:00
            SPY Log Return
Date                      
1998-12-23        0.020757
1998-12-24       -0.004321
1998-12-28       -0.002550
1998-12-29        0.015708
1998-12-30       -0.008077
...                    ...
2020-12-23        0.000898
2020-12-24        0.003883
2020-12-28        0.008554
2020-12-29       -0.001910
2020-12-30        0.001426

[5541 rows x 1 columns]

            SPY Log Return
2001-05-07       -0.008676
2001-05-08       -0.000475
2001-05-09       -0.004209
2001-05-10        0.002940
2001-05-11       -0.006928
...                    ...
2020-11-11        0.007401
2020-11-12       -0.009748
2020-11-13        0.013750
2020-11-16        0.012405
2020-11-17       -0.005393

[4916 rows x 1 columns]
Type: ETF, Ticker: SPY, Period: All Periods, Volatility: 0.19601497477889382, First Date: 2001-05-07 00:00:00, Last Date: 2020-11-17